In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import re
import shutil

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Code")
import cleaning_tweets
from cleaning_tweets import preprocess, preprocess_hg
os.chdir("/content/drive/MyDrive/Data")
path_txt = "/content/drive/MyDrive/Data/wordcloud/"

!pip install wordle==0.2.1
from wordle import Wordle, export_wordcloud
import pathlib

WordCloud for hashtags

In [ ]:
df_mini = pd.read_csv('data_with_labels_mini.csv', header = None, dtype = str, encoding = 'utf-8-sig', engine='python')
df_mini.columns = ['date', 'id', 'text', 'hashtags', \
                                    'state', 'county and city',\
                                    'retweet_count', 'favorite_count',\
                                    'text_id', 'st_labels', 'st_scores', 'aspects']

In [ ]:
txtfile = open('COVID19_AIDR_Keywords.txt')
lines = txtfile.readlines()
ids = []
for line in lines:
    line = line.split(',')
    for i in line:
        i = re.sub(r'[^a-zA-Z]', '', i)
        if i != '':
            i = i.lower()
            ids.append(i.strip())

In [ ]:
def encodetoken_hg(tokens, keyword):
    comment_words = ''
    if keyword == 0:
        # remove covid related tags
        token_lst = [token.strip() for token in tokens if token.strip() not in ids]
        comment_words += " ".join(token_lst)+" "
    else:
        token_lst = [token.strip() for token in tokens]
        comment_words += " ".join(token_lst)+" "
    return comment_words

In [ ]:
# hashtags in 4million data
df1 = df_mini.copy()
df1.dropna(subset=['hashtags'], axis=0, how='any', inplace=True)
df1.reset_index(inplace=True, drop=True)
df1.loc[:, 'preprocess_text'] = df1['hashtags'].apply(preprocess_hg)
df1.loc[:, 'text_nonstop_cvd'] = df1['preprocess_text'].apply(lambda r: encodetoken_hg(r, 1))
df1.loc[:, 'text_nonstop_noncvd'] = df1['preprocess_text'].apply(lambda r: encodetoken_hg(r, 0))
df1['text_nonstop_cvd'].to_csv('hashtags_4m_cvd.txt', header=None, index=False, sep="\t")
df1['text_nonstop_noncvd'].to_csv('hashtags_4m_noncvd.txt', header=None, index=False, sep="\t")
w = Wordle(random_state=5678)
w.generate_from_file('hashtags_4m_cvd.txt', outfile='hashtags_4m_cvd.png')
w.generate_from_file('hashtags_4m_noncvd.txt', outfile='hashtags_4m_noncvd.png')
os.remove('hashtags_4m_cvd.txt')
os.remove('hashtags_4m_noncvd.txt')
shutil.move('hashtags_4m_cvd.png', path_txt+ 'hashtags_4m_cvd.png')
shutil.move('hashtags_4m_noncvd.png', path_txt+ 'hashtags_4m_noncvd.png')

WordCloud for different aspects

In [ ]:
df = pd.read_csv('ap_textid_mini.csv', dtype = str, encoding = 'utf-8-sig', engine='python')

In [ ]:
def encodetoken(tokens):
    comment_words = ''
    token_lst = [token for token in tokens]
    comment_words += " ".join(token_lst)+" "
    return comment_words

In [ ]:
w = Wordle(random_state=5678)
labels = ['negative', 'positive']
for idx in range(0, 9):
    df1 = df[df['aspects_idx']==str(idx)].copy()
    for i in range(2):
        df2 = df1[df1['st_labels']==str(i)].copy()
        df2.loc[:, 'preprocess_text'] = df2['text'].apply(preprocess)
        df2.loc[:, 'text_nonstop'] = df2['preprocess_text'].apply(lambda r: encodetoken(r))
        filename = 'aspect_' + labels[i] + str(idx)
        df2['text_nonstop'].to_csv(filename +'.txt', header=None, index=False, sep="\t")
        w.generate_from_file(filename + '.txt', outfile = filename + '.png')
        os.remove(filename + '.txt')
        shutil.move(filename + '.png', path_txt + filename + '.png')